In [3]:
# 경고(warning) 비표시
import warnings
warnings.filterwarnings('ignore')

# 제1장 웹에서 주문 수를 분석하는 테크닉 10
- 목적 : 어떤 기업 쇼핑몰 사이트의 상품 주문 수의 추세를 분석하고 판매량 개선의 방향을 찾는 것

데이터분석가 역량은 데이터를 어떻게 연결해서 활용하는 지 보여주는 부분

## tech1 : 데이터를 읽어들이자
해당 쇼핑몰의 주요 품목은 컴퓨터이다

- 데이터 정보
    - customer_master.csv : 고객 데이터, 이름, 성별 등
    - item_master.csv : 상품 데이터
    - transaction_1.csv : 구매내역 데이터
    - transaction_2.csv : 구매내역 분할 데이터
    - transaction_detail_1.csv : 구매내역 상세 데이터
    - transaction_detail_2.csv : 구매내역 분할 상세 데이터


데이터베이스에 들어있는 데이터를 시스템에서 가져올 때는 월별이나 1,000건 단위로 분할하는 경우가 많음. 이 경우에는 데이터를 결합해서 사용해야 한다.

In [10]:
import pandas as pd

# 고객 상세 정보
customer_master = pd.read_csv("1장_dataset/customer_master.csv")
customer_master.head(3)

,customer_id,customer_name,registration_date,email,gender,age,birth,pref
0,IK152942,김서준,2019-01-01 0:25,hirata_yuujirou@example.com,M,29,1990-06-10,대전광역시
1,TS808488,김예준,2019-01-01 1:13,tamura_shiori@example.com,F,33,1986-05-20,인천광역시
2,AS834628,김도윤,2019-01-01 2:00,hisano_yuki@example.com,F,63,1956-01-02,광주광역시


In [12]:
# 상품명, 상품 단가 정보
item_master = pd.read_csv("1장_dataset/item_master.csv")
item_master.head()

,item_id,item_name,item_price
0,S001,PC-A,50000
1,S002,PC-B,85000
2,S003,PC-C,120000
3,S004,PC-D,180000
4,S005,PC-E,210000


In [14]:
# 거래 데이터 정보 (고객, 시간, 거래id, 금액)
transaction_1 = pd.read_csv("1장_dataset/transaction_1.csv")
transaction_1.head()

,transaction_id,price,payment_date,customer_id
0,T0000000113,210000,2019-02-01 01:36:57,PL563502
1,T0000000114,50000,2019-02-01 01:37:23,HD678019
2,T0000000115,120000,2019-02-01 02:34:19,HD298120
3,T0000000116,210000,2019-02-01 02:47:23,IK452215
4,T0000000117,170000,2019-02-01 04:33:46,PL542865


In [15]:
# 거래시 구매 상품, 수량
transaction_detail_1 = pd.read_csv("1장_dataset/transaction_detail_1.csv")
transaction_detail_1.head()

,detail_id,transaction_id,item_id,quantity
0,0,T0000000113,S005,1
1,1,T0000000114,S001,1
2,2,T0000000115,S003,1
3,3,T0000000116,S005,1
4,4,T0000000117,S002,2


데이터를 확인하였다! 그렇다면, 이제는 어떤 식으로 접근하는 것이 좋을까?

> **목적에 상관없이 우선은! 데이터 전체를 파악하는 것이 중요**
- 되도록 상세하게 나와있는 데이터에 맞춰서 데이터 가공하기
- 여기에서는 transaction_detail_1 에 해당

<br>

**transaction_detail_1 기준으로 생각할 경우 2가지의 데이터 가공이 필요**
1) transaction_detail_1, detail_2 / transaction 데이터들을 세로로 결합 (유니언)
2) detail을 기준으로 다른 데이터들을 가로로 결합 (조인))


### tech2 : 데이터를 결합(union)해 보자

In [27]:
transaction_2 = pd.read_csv("1장_dataset/transaction_2.csv")

# union
transaction = pd.concat([transaction_1, transaction_2], ignore_index = True) # ingore_index True : 기존 index 무시
transaction.head(3)

,transaction_id,price,payment_date,customer_id
0,T0000000113,210000,2019-02-01 01:36:57,PL563502
1,T0000000114,50000,2019-02-01 01:37:23,HD678019
2,T0000000115,120000,2019-02-01 02:34:19,HD298120


union은 데이터를 행 방향으로 늘리는 것이기에 **개수의 변화를 확인**해야한다.

In [28]:
print(len(transaction_1))
print(len(transaction_2))
print(len(transaction))

5000
1786
6786


In [29]:
transaction_detail_2 = pd.read_csv("1장_dataset/transaction_detail_2.csv")

# union
transaction_detail = pd.concat([transaction_detail_1, transaction_detail_2], ignore_index = True) # ingore_index True : 기존 index 무시
transaction_detail.head(3)

,detail_id,transaction_id,item_id,quantity
0,0,T0000000113,S005,1
1,1,T0000000114,S001,1
2,2,T0000000115,S003,1


## tech3 : 매출 데이터끼리 결합(조인)해 보자

데이터를 조인할 때는 기준이 되는 데이터를 정확하게 결정 (가장 상세한 데이터를 기준으로 설정)
- 추가하고 싶은 컬럼
- 공통되는 컬럼

In [31]:
join_data = pd.merge(transaction_detail, transaction[["transaction_id", "payment_date", "customer_id"]],
                     on = "transaction_id", how= "left")
join_data.head()

,detail_id,transaction_id,item_id,quantity,payment_date,customer_id
0,0,T0000000113,S005,1,2019-02-01 01:36:57,PL563502
1,1,T0000000114,S001,1,2019-02-01 01:37:23,HD678019
2,2,T0000000115,S003,1,2019-02-01 02:34:19,HD298120
3,3,T0000000116,S005,1,2019-02-01 02:47:23,IK452215
4,4,T0000000117,S002,2,2019-02-01 04:33:46,PL542865


In [32]:
print(len(transaction_detail))
print(len(transaction))
print(len(join_data))

7144
6786
7144


### tech4. 마스터데이터를 결합(조인) 해 보자

In [34]:
join_data = pd.merge(join_data, customer_master, on = "customer_id", how = "left")
join_data = pd.merge(join_data, item_master, on = 'item_id', how = "left")
join_data.head()

,detail_id,transaction_id,item_id,quantity,payment_date,customer_id,customer_name,registration_date,email,gender,age,birth,pref,item_name,item_price
0,0,T0000000113,S005,1,2019-02-01 01:36:57,PL563502,김태경,2019-01-07 14:34,imoto_yoshimasa@example.com,M,30,1989-07-15,대전광역시,PC-E,210000
1,1,T0000000114,S001,1,2019-02-01 01:37:23,HD678019,김영웅,2019-01-27 18:00,mifune_rokurou@example.com,M,73,1945-11-29,서울특별시,PC-A,50000
2,2,T0000000115,S003,1,2019-02-01 02:34:19,HD298120,김강현,2019-01-11 8:16,yamane_kogan@example.com,M,42,1977-05-17,광주광역시,PC-C,120000
3,3,T0000000116,S005,1,2019-02-01 02:47:23,IK452215,김주한,2019-01-10 5:07,ikeda_natsumi@example.com,F,47,1972-03-17,인천광역시,PC-E,210000
4,4,T0000000117,S002,2,2019-02-01 04:33:46,PL542865,김영빈,2019-01-25 6:46,kurita_kenichi@example.com,M,74,1944-12-17,광주광역시,PC-B,85000
